In [0]:
from pyspark.streaming import StreamingContext
import json

In [0]:
ssc = StreamingContext(sc, 1)

In [0]:
lines = ssc.socketTextStream("localhost",3340)

In [0]:
def aggr_min(new_values, total_min):
    return min(new_values + [(total_min or new_values[0])])

In [0]:
from pyspark.sql import Row

In [0]:
def process_rdd(time,rdd):
    if rdd.count() > 0:
        row_rdd = rdd.map(lambda w : Row(stock=w[0],min_price=w[1]))
        min_df = spark.createDataFrame(row_rdd)
        min_df.createOrReplaceTempView("min_prices")

In [0]:
def extract_json(js):
    json_obj = json.loads(js)
    return (json_obj['stock'],json_obj['prezzo'])

In [0]:
lines.flatMap(lambda txt : txt.split("\n")) \
    .map(extract_json) \
    .updateStateByKey(aggr_min) \
    .foreachRDD(process_rdd)

In [0]:
sc.setCheckpointDir("/home/spark_checkpoints")

In [0]:
ssc.start()

In [0]:
%sql


select * from min_prices

stock,min_price
Azienda 1,73.44879079573735
Azienda 2,89.04040625251616
Azienda 3,18.715467870600076
